In [ ]:
print('test')

# Sheet

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

plt.style.use('dark_background')
figure(figsize=(16, 8), dpi=60)
plt.rcParams["figure.figsize"] = (18,5)

from os import listdir
from os.path import isfile, join
from pathlib import *
import pandas as pd

from fin.LoadKnab import parse_csv_knab

data_path = Path('./data')

fin_headers = ['date', 'year', 'month', 'credit_or_debet', 'amount', 'credit', 'debet', 'i_number', 'i_name', 'e_number', 'e_name', 'internal_or_external', 'e_type', 'e_official_name', 'description', 'payment_type']

pri_inkomen, pri_boodschappen, pri_fun, pri_overig, pri_abbo, pri_stack, pri_stack_kaakoperatie, huis_inkomen_en_lasten, huis_klussen, huis_huishoudelijk, huis_spaar = [None] * 11
df_pri_inkomen, df_pri_boodschappen, df_pri_fun, df_pri_overig, df_pri_abbo, df_pri_stack, df_pri_stack_kaakoperatie, df_huis_inkomen_en_lasten, df_huis_klussen, df_huis_huishoudelijk, df_huis_spaar = [None] * 11

labels = {
    'NL25KNAB0604860706': 'pri_inkomen',
    'NL60KNAB0604871716': 'pri_boodschappen',
    'NL54KNAB0604873182': 'pri_fun',
    'NL55KNAB0604872291': 'pri_overig',
    'NL08KNAB0604887493': 'pri_abbo',
    '57347285': 'pri_stack',
    '57441905': 'pri_stack_kaakoperatie',
    'NL02KNAB0407458360': 'huis_inkomen_en_lasten',
    'NL45KNAB0407464085': 'huis_klussen',
    'NL13KNAB0407465243': 'huis_huishoudelijk',
    '57857884': 'huis_spaar',
    'NL55INGB0798628839': 'huis_hypotheek',
    'NL06RABO0154599808': 'pri_oud',
    '57673844': 'pri_stack_oud',
    '60010259': 'pri_stack_oud2',
}

fin_data = []
fin_pandas = []
csv_paths = [f for f in listdir(data_path) if isfile(join(data_path, f))]

for i, cvs_path in enumerate(csv_paths):

    path = data_path / cvs_path
    account_number, account_type, year = parse_csv_knab(cvs_path)

    label = labels[account_number]

    s = label.split('_')
    account_name, account_category = (s.pop(0), '_'.join(s))

    df = pd.read_csv(path, skiprows=1, sep=';', decimal=',')
    df['Bedrag'] = df['Bedrag'].astype(float)

    df['Transactiedatum'] = pd.to_datetime(df['Transactiedatum'], dayfirst=True)
    df['Year'] = df['Transactiedatum'].dt.year
    df['Month'] = df['Transactiedatum'].dt.month

    def transform(c):
        c['Credit'] = c['Bedrag'] if c['CreditDebet'] == 'C' else 0
        c['Debet'] = c['Bedrag'] if c['CreditDebet'] == 'D' else 0
        c['RekeningnummerNaam'] = labels[c['Rekeningnummer']] if c['Rekeningnummer'] in labels else '?'
        c['TegenrekeningnummerNaam'] = labels[c['Tegenrekeningnummer']] if c['Tegenrekeningnummer'] in labels else c['Tegenrekeninghouder']
        c['InternalExternal'] = 'I' if c['Tegenrekeningnummer'] in labels else 'E'
        c['e_type'] = ('P' if c['Tegenrekeningnummer'].startswith('NL') else 'S') if c['InternalExternal'] == 'I' else None

        return c

    df = df.apply(transform, axis=1)
    df = df[['Transactiedatum', 'Year', 'Month', 'CreditDebet', 'Bedrag', 'Credit', 'Debet', 'Rekeningnummer', 'RekeningnummerNaam', 'Tegenrekeningnummer', 'TegenrekeningnummerNaam', 'InternalExternal', 'e_type', 'Tegenrekeninghouder', 'Omschrijving', 'Betaalwijze']]
    df.set_axis(fin_headers, axis=1, inplace=True)

    fin_pandas.append(df)

    account_info = {
        'path': path,
        'df': df,
        'account_name': account_name,
        'account_category': account_category,
        'account_number': account_number,
        'account_type': account_type,
        'year': year,
    }

    account_info = Struct(account_info)
    locals().update({label: account_info, f'df_{label}': df})
    fin_data.append(account_info)



IndexError: list index out of range

<Figure size 960x480 with 0 Axes>

In [ ]:
df = df_pri_inkomen
df = df.loc[df['credit_or_debet'] == 'C']
df = df.loc[df['internal_or_external'] == 'E']

print(df['e_name'].value_counts())
print(df['e_number'].value_counts())
print(df['payment_type'].value_counts())

df

# Inkomen

In [ ]:
df = df_pri_inkomen
df = df.loc[df['credit_or_debet'] == 'C']
df = df.loc[df['internal_or_external'] == 'E']

print(df['e_name'].value_counts())

df

In [ ]:
df_per_crediteur = df.groupby('e_name').agg({'credit': ['sum'], 'amount': 'sum'})

labels = df_per_crediteur.index

print(labels)

df_per_crediteur['credit'].iloc[:, 0]

In [ ]:
df = df_pri_inkomen
df = df.loc[df['credit_or_debet'] == 'C']
df = df.loc[df['internal_or_external'] == 'E']

df_per_maand = df.groupby('month').agg({'credit': ['sum'], 'amount': 'sum'})

labels = df_per_maand.index
values = list(df_per_maand['credit'].iloc[:, 0])

# labels = df['month']
# values = df['credit']

maand_labels = ('Januari', 'Februari', 'Maart', 'April', 'Mei', 'Juni', 'Juli', 'Augustus', 'September', 'Oktober', 'November', 'December')
maand_nummers = (1,2,3,4,5,6,7,8,9,10,11,12)

fig, ax = plt.subplots()

hbars = ax.barh(labels, values, align='center')
ax.set_yticks(maand_nummers, labels=maand_labels)

ax.set_xlabel('Euro')
ax.set_ylabel('Maanden')
ax.set_title('2021 Inkomen per maand')

# Label with specially formatted floats
# ax.bar_label(hbars, fmt='%.2f')
ax.bar_label(hbars, fmt=' €%.2f')
ax.set_xlim(right=(max(values)*1.1))  # adjust xlim to fit labels

plt.show()

In [ ]:
df = df_pri_inkomen
df = df.loc[df['credit_or_debet'] == 'C']

df_per_maand = df.groupby('month').agg({'credit': ['sum'], 'amount': 'sum'})

labels = df_per_maand.index
values = list(df_per_maand['amount'].iloc[:, 0])

# labels = df['month']
# values = df['credit']

maand_labels = ('Januari', 'Februari', 'Maart', 'April', 'Mei', 'Juni', 'Juli', 'Augustus', 'September', 'Oktober', 'November', 'December')
maand_nummers = (1,2,3,4,5,6,7,8,9,10,11,12)

fig, ax = plt.subplots()

hbars = ax.barh(labels, values, align='center')
ax.set_yticks(maand_nummers, labels=maand_labels)

ax.set_xlabel('Euro')
ax.set_ylabel('Maanden')
ax.set_title('2021 Inkomen per maand')

# Label with specially formatted floats
# ax.bar_label(hbars, fmt='%.2f')
ax.bar_label(hbars, fmt=' €%.2f')
ax.set_xlim(right=(max(values)*1.1))  # adjust xlim to fit labels

plt.show()

# Uitgaven

In [ ]:
df = df_pri_inkomen
df = df.loc[df['credit_or_debet'] == 'D']
df = df.loc[df['e_type'] != 'S']

df_per_category = df.groupby(['e_name', 'internal_or_external']) \
                             .agg({'amount': ['count',  'sum', ('per_month', lambda v: v.sum() / 12), 'mean', 'min', 'max']}) \
                             .sort_values(by=[('amount', 'count')], ascending=False)
df_per_category[[('amount','sum')]].plot.barh()

print("totale uitgaven €", sum(df_per_category[('amount','sum')]))

df_per_category

In [ ]:
df = df_pri_inkomen
df = df.loc[df['credit_or_debet'] == 'D']

print(df['e_name'].value_counts())

df

In [ ]:
df_per_account = df.groupby('i_name').agg({'amount': 'sum'})


labels = df_per_account.index

print(labels)

values = list(df_per_account['amount'])
values

In [ ]:
df_per_maand = df.groupby('month').agg({'credit': ['sum'], 'amount': 'sum'})

labels = df_per_maand.index
values = list(df_per_maand['amount'].iloc[:, 0])

# labels = df['month']
# values = df['credit']

maand_labels = ('Januari', 'Februari', 'Maart', 'April', 'Mei', 'Juni', 'Juli', 'Augustus', 'September', 'Oktober', 'November', 'December')
maand_nummers = (1,2,3,4,5,6,7,8,9,10,11,12)

fig, ax = plt.subplots()

hbars = ax.barh(labels, values, align='center')
ax.set_yticks(maand_nummers, labels=maand_labels)

ax.set_xlabel('Euro')
ax.set_ylabel('Maanden')
ax.set_title('2021 Inkomen per maand')

# Label with specially formatted floats
# ax.bar_label(hbars, fmt='%.2f')
ax.bar_label(hbars, fmt=' €%.2f')
ax.set_xlim(right=(max(values)*1.1))  # adjust xlim to fit labels

plt.show()

In [ ]:
df = df_pri_inkomen
df = df.loc[df['credit_or_debet'] == 'D']
df = df.loc[df['internal_or_external'] == 'I']
df = df.loc[df['e_type'].isin(['P', 'S'])]

df_per_maand = df.groupby(['month', 'e_name']).agg({'amount': ['sum', 'mean', 'min', 'max']})

df_per_maand.plot.bar()
df_per_maand

In [ ]:
df.groupby(['e_name']).agg({'amount': ['sum', ('per_month', lambda v: v.sum() / 12), 'mean']}).plot.barh()

df_per_category = df.groupby(['e_name']).agg({'amount': ['sum', ('per_month', lambda v: v.sum() / 12), 'mean', 'min', 'max']})
df_per_category.plot.barh()
df_per_category

In [ ]:
df_pri_inkomen

## Beugel

In [ ]:
df = df_pri_overig

old_costs = [
    130.0,
    5.44,
    58.12,
    8.15,
    240.0,
    241.19,
    41.44,
    41.44,
    41.44,
    41.44,
    45.7,
    93.03,
    41.44,
    41.44
]

old_costs_total = sum(old_costs)

print("old costs: €", old_costs_total)

df_beugel = df.loc[df['e_name'] == 'Infomedics B.V.']

print(df_beugel['amount'])

costs = sum(df_beugel['amount'])

costs = costs + old_costs_total

prediction = 12 * 42.77

inzet_kosten = 240.0 + 241.19

print("beugelkosten: €", costs)
print("wachten tot kaakoperatie: €", prediction)
print("beugel plaatsen: €", inzet_kosten)
print("beugel verwijderen en plaatsten: €", inzet_kosten)


## abbo

In [ ]:
df = df_pri_abbo
df = df.loc[df['credit_or_debet'] == 'D']

print(df['e_name'].value_counts())

df

In [ ]:
df.groupby(['e_name']).agg({'amount': ['sum', ('per_month', lambda v: v.sum() / 12), 'mean']}).plot.barh()

df_per_category = df.groupby(['e_name']).agg({'amount': ['sum', ('per_month', lambda v: v.sum() / 12), 'mean', 'min', 'max']})
df_per_category.plot.barh()
df_per_category

## Sauna

In [ ]:
# df = df_pri_overig 
df = df_pri_fun 

search_term = 'thermen'

search = df[df['e_official_name'].str.contains(search_term, na=False, case=False)]
search2 = df[df['description'].str.contains(search_term, na=False, case=False)]

c = ['year', 'month','amount', 'e_official_name', 'description']
print('search1')
print(search[c])
print('search2')
print(search2[c])

df

## Overig

In [ ]:
df = df_pri_overig
df = df.loc[df['credit_or_debet'] == 'D']

print(df['e_name'].value_counts())

df

In [ ]:
df_per_category = df.groupby(['e_name']) \
                             .agg({'amount': ['count',  'sum', ('per_month', lambda v: v.sum() / 12), 'mean', 'min', 'max']}) \
                             .sort_values(by=[('amount', 'count')], ascending=False)
df_per_category[('amount', 'sum')].plot.barh()
df_per_category

In [ ]:
name = []
count = []
amount = []

aggregates = {
    5: [0, 0],
    10: [0, 0],
    25: [0, 0],
    50: [0, 0],
    100: [0, 0],
}
l = list(aggregates)
first_aggregates = l
last_aggregate = l[-1] + 1
aggregates.update({last_aggregate: [0, 0]})

def aggregate_small_values(c):
    n = c.name    
    s = c[('amount', 'sum')]
    c = c[('amount', 'count')]

    if c <= 2 and s < 151:
    
        if s < last_aggregate:
            for a in first_aggregates:
                if s <= a:
                    aggregates[a][0] += c
                    aggregates[a][1] += s
                    break
        else:
            aggregates[last_aggregate][0] += c
            aggregates[last_aggregate][1] += s
    else:
        name.append(n)
        count.append(c)
        amount.append(s)

df_per_category.apply(aggregate_small_values, axis=1)

for a in first_aggregates:
    name.append(f'{a} and smaller')
    count.append(aggregates[a][0])
    amount.append(aggregates[a][1])

name.append(f'{last_aggregate} and bigger')
count.append(aggregates[last_aggregate][0])
amount.append(aggregates[last_aggregate][1])

df_agg = pd.DataFrame(index=name, data={'count': count, 'amount': amount})

plt.rcParams["figure.figsize"] = (18,10)
df_agg.plot.barh(title='Overig 2021')
plt.rcParams["figure.figsize"] = (18,5)

df_agg

# pin

In [ ]:
df_pin = df[df['amount'] >= 149]
df_pin = df_pin[df_pin['payment_type'] == 'Geldautomaat']

b = sum(df_pin['amount']) - 200
pm = round(b / 12, 2)

print('Pin: €', b)
print('P.M.: €', )
print('% van I: ', round(pm / 2100 * 100), '%')

df_pin

### Boetes!!!

In [ ]:
# df_boete = df.loc[df['e_name'] == 'cjib']

search_term = 'cjib'
df_boete = df[df['e_official_name'].str.contains(search_term, na=False, case=False)]

print(df_boete['amount'])

costs = sum(df_boete['amount'])

print("Boetes: €", costs)
print("Per maand: €", round(costs / 12, 2))

In [ ]:
# df[df['description'].str.contains(search_term, na=False, case=False)]

df_boete = df.loc[df['description'] == '8062542246114392']
df_boete

## Boodschappen

In [ ]:
df = df_pri_boodschappen
df = df.loc[df['credit_or_debet'] == 'D']

print(df['e_name'].value_counts())

df

In [ ]:
budget = 250

df_per_month = df.groupby(['month']) \
                             .agg({'amount': ['count',  'sum', ('per_day', lambda v: v.sum() / 30), 'mean', 'min', 'max']})


%matplotlib inline

fig, axes = plt.subplots(figsize=(10,8))



plt.plot([-1, 12], [budget, budget], color='red')

ax = df_per_month[('amount', 'sum')].plot.bar(ax=axes, yerr=df_per_month[('amount','per_day')], title='Budget boodschappen 2021')

plt.show()

df_per_month

In [ ]:
df_per_category = df.groupby(['e_name']) \
                             .agg({'amount': ['count',  'sum', ('per_month', lambda v: v.sum() / 12), 'mean', 'min', 'max']}) \
                             .sort_values(by=[('amount', 'count')], ascending=False)

df_per_category

In [ ]:
name = []
count = []
amount = []

aggregates = {
    5: [0, 0],
    10: [0, 0],
    25: [0, 0],
    50: [0, 0],
}
l = list(aggregates)
first_aggregates = l
last_aggregate = l[-1] + 1
aggregates.update({last_aggregate: [0, 0]})

def aggregate_small_values(c):
    n = c.name    
    s = c[('amount', 'sum')]
    c = c[('amount', 'count')]

    if c <= 2:
    
        if s < last_aggregate:
            for a in first_aggregates:
                if s <= a:
                    aggregates[a][0] += c
                    aggregates[a][1] += s
                    break
        else:
            aggregates[last_aggregate][0] += c
            aggregates[last_aggregate][1] += s
    else:
        name.append(n)
        count.append(c)
        amount.append(s)

df_per_category.apply(aggregate_small_values, axis=1)

for a in first_aggregates:
    name.append(f'{a} and smaller')
    count.append(aggregates[a][0])
    amount.append(aggregates[a][1])

name.append(f'{last_aggregate} and bigger')
count.append(aggregates[last_aggregate][0])
amount.append(aggregates[last_aggregate][1])

df_agg = pd.DataFrame(index=name, data={'count': count, 'amount': amount})

df_agg.plot.barh(title='Boodschappen 2021')

df_agg

## Huis

In [ ]:
df = df_huis_inkomen_en_lasten
df = df.loc[df['credit_or_debet'] == 'D']

print(df['e_name'].value_counts())

df

In [ ]:
budget = 250

df_per_month = df.groupby(['month']) \
                             .agg({'amount': ['count',  'sum', ('per_day', lambda v: v.sum() / 30), 'mean', 'min', 'max']})


%matplotlib inline

fig, axes = plt.subplots(figsize=(10,8))



# plt.plot([-1, 12], [budget, budget], color='red')

ax = df_per_month[('amount', 'sum')].plot.bar(ax=axes, yerr=df_per_month[('amount','per_day')], title='Huis lasten 2021')

plt.show()

df_per_month

In [ ]:
df_per_category = df.groupby(['e_name']) \
                             .agg({'amount': ['count',  'sum', ('per_month', lambda v: v.sum() / 12), 'mean', 'min', 'max']}) \
                             .sort_values(by=[('amount', 'count')], ascending=False)

df_per_category[('amount','sum')].plot.barh(title='Huis lasten 2021')

df_per_category

# Sheet 2

In [ ]:
import re

re.sub(' +', ' ', 'The     quick brown    fox')

In [ ]:
d = {'name': ['a', 'a', 'b', 'b', 'b', 'c', 'd', 'e'], 'amount': [2, 5, 2, 3, 7, 2, 4, 1]}
df = pd.DataFrame(data=d)

df_per_category = df.groupby(['name']) \
                             .agg({'amount': ['count',  'sum']}) \
                             .sort_values(by=[('amount', 'count')], ascending=False)

df_per_category[('amount', 'sum')].plot.barh()
df_per_category

In [ ]:
aggregates = {
    5: 0,
    10: 0,
    50: 0,
    100: 0,
}

for a in aggregates:
    print(a, aggregates[a])

In [ ]:
# 7.58 <= 50 and not 7.58 > 50
# 50 and smaller

# 7.58 <= 50 #and not 7.58 > 50

7.58 > 50